# **PQC Implementation**

* Explored solutions to the problem of secure storage of data.
* Implemented data encryption and suitable key management (secure way to share key) architecture in software.
* Produced the encrypted backup of files and folders.
* Implemented encryption/decryption of files using separate keys for two different users.


### **Explanation**

*   Key Management: Kyber-KEM
*   Encrpted Backup: Google Drive
*   Encryption/Decryption: AES
*   Digital Signature: SPHINCS+

AES in GCM mode ensures confidentiality and authenticity while SPHINCS+ ensures the integrity.

In [ ]:
# mount google drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/MyDrive/Task2')

## **Kyber Key Sharing**

A Key-Encapsulation Mechanism (KEM) is used to securely share key between two parties.

References:

https://pypi.org/project/kyber-py/

https://github.com/GiacomoPope/kyber-py

In [ ]:
!pip install kyber-py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 33.1 MB/s eta 0:00:00


In [ ]:
from kyber import Kyber512

def kyber_key_sharing():
  print("--------------------------")
  print("*** User A Key Sharing ***")
  print("-------------------------- \n")

  # kyber key sharing for user A
  print("A generates public-private key pair \n")

  # public (pkA) and private (skA) key
  pkA, skA = Kyber512.keygen()

  print("A sends public key over the internet \n")
  print("Public Key: ", pkA)

  print("B recieves A's public key, generates cipher text and shared key \n")

  cA, keyA = Kyber512.enc(pkA)

  print("B sends cipher text to A \n")
  print("Cipher Text: ", cA)

  print("A recieves cipher text, generates same shared key \n")

  _keyA = Kyber512.dec(cA, skA)

  if keyA == _keyA:
    print("A's key sharing successful \n")

  else:
    print("A's key sharing failed \n")

  print("--------------------------")
  print("*** User B Key Sharing ***")
  print("-------------------------- \n")

  # kyber key sharing for user
  print("B generates public-private key pair \n")

  # public (pkB) and private (skB) key
  pkB, skB = Kyber512.keygen()

  print("B sends public key over the internet \n")
  print("Public Key: ", pkB)


  print("A recieves B's public key, generates cipher text and shared key \n")

  cB, keyB = Kyber512.enc(pkB)

  print("A sends cipher text to B \n")
  print("Cipher Text: ", cB)


  print("B recieves cipher text, generates same shared key \n")

  _keyB = Kyber512.dec(cB, skB)

  if keyB == _keyB:
    print("B's Key sharing successful \n")

  else:
    print("B's Key sharing failed \n")

  return keyA, keyB

## **AES Encryption Function**

In [ ]:
!pip install pycryptodomex==3.18.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 32.1 MB/s eta 0:00:00


**SPHINCS+ Digital Signature**

Reference:

https://github.com/sphincs/pyspx

In [ ]:
!pip install pyspx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 20.5 MB/s eta 0:00:00


In [ ]:
import pyspx.shake_128f as sphincs
import os, binascii

# key generation
seed = os.urandom(sphincs.crypto_sign_SEEDBYTES)
public_key, secret_key = sphincs.generate_keypair(seed)

print("Public key: ", binascii.hexlify(public_key))
print("Private key: ", binascii.hexlify(secret_key))


Public key:  b'1e74c805841f0fb50fe9cc7fc18f7595dff8815c966b959878143d505740af4b'
Private key:  b'b749e6565914047e1590750d3a6ce1c2eddec6f01aa84e835d09b2b2222c3e561e74c805841f0fb50fe9cc7fc18f7595dff8815c966b959878143d505740af4b'


In [ ]:
from Cryptodome.Cipher import AES
import os
import base64

# definine backup folder path
BACKUP_FOLDER = '/content/drive/MyDrive/Task2/backup/'

os.makedirs(BACKUP_FOLDER, exist_ok=True)

# open the file to encrypt in "binary read mode" so the file content is read exactly they are
def encrypt_file(filename, user, key):
  with open(filename, 'rb') as file:
    plain = file.read()

  # generate SPHINCS+ digital signature
  signature = sphincs.sign(plain, secret_key)
  print("Signature: ", binascii.hexlify(signature)) # converts to hexadecimal format
  print("")

  # initialize AES in GCM mode with the Kyber key
  aes_cipher = AES.new(key, AES.MODE_GCM) # initialises cipher object
  nonce = aes_cipher.nonce
  ciphertext, tag = aes_cipher.encrypt_and_digest(plain) # encrypts plaintext

  # output file saved in backup
  output_filename = os.path.join(BACKUP_FOLDER, os.path.basename(filename) + '.enc')

  # choose keys according to user
  if user == 'A':
    label = 'keyA'
  elif user == 'B':
    label = 'keyB'
  else:
    print("Invalid user \n")
    return

  # open the file in "binary write mode"
  with open(output_filename, 'wb') as output_file:
    encoded_label = base64.b64encode(label.encode()) + b'\n'
    output_file.write(encoded_label)
    output_file.write(nonce)
    output_file.write(tag)
    output_file.write(signature)
    output_file.write(ciphertext)

  print(f"Encryption Successfull \n")
  print(f"Encrypted file saved as {output_filename} \n")


## **AES Decryption Function**

In [ ]:
from Cryptodome.Cipher import AES
import os

# define local folder path
LOCAL_FOLDER = '/content/'
os.makedirs(LOCAL_FOLDER, exist_ok=True)

# function to decrypt a file
def decrypt_file(filename, keyA, keyB, public_key):
  with open(filename, 'rb') as file:
    encoded_label = file.readline().strip()
    label = base64.b64decode(encoded_label).decode()
    nonce = file.read(16)
    tag = file.read(16)
    signature = file.read(sphincs.crypto_sign_BYTES)  # SPHINCS+ signature
    ciphertext = file.read()

  # initialize the AES cipher based on the label
  if label == 'keyA':
    aes_cipher = AES.new(keyA, AES.MODE_GCM, nonce=nonce)
  elif label == 'keyB':
    aes_cipher = AES.new(keyB, AES.MODE_GCM, nonce=nonce)
  else:
    print("Invalid label \n")
    return

  try:
    plain = aes_cipher.decrypt_and_verify(ciphertext, tag)
    print("Decryption successful \n")

  except ValueError as e:
    print("Decryption failed: ", e)
    return

  # sphincs+ verification
  if sphincs.verify(plain, signature, public_key):
    print("Digital Signature verified")
    print("Data has not been tampered \n")

  else:
    print("Digital Signature not verified \n")
    print("Data has been tampered \n")
    return

  # define the decrypted output filename
  output_filename = os.path.join(LOCAL_FOLDER, os.path.basename(filename)[:-4] + '.dec')

  # save the decrypted data to a new file
  with open(output_filename, 'wb') as output_file:
    output_file.write(plain)

  print(f"Decrypted file saved as {output_filename} \n")


## **Main Program**

In [ ]:
import os

def main():
  print("------------------------------------")
  print("*** FILE ENCRYPTION & DECRYPTION *** ")
  print("------------------------------------\n")

  print("Kyber Key Sharing \n")

  # kyber key sharing
  keyA, keyB = kyber_key_sharing()

  print("Kyber Key Sharing Successful \n")

  while True:
    print("---------------")
    print("*** SIGN IN *** ")
    print("---------------\n")

    print("Choose 'A' or 'B' \n")

    user = input("User : ").strip().upper()
    print("")

    if user not in ['A', 'B']:
      print("Invalid user \n")
      continue

    print("------------------------------------")
    print("*** FILE ENCRYPTION & DECRYPTION *** ")
    print("------------------------------------\n")

    print("Encryption : 'e'  |  Decryption : 'd'  |  Quit : 'q' \n")

    choice = input("Option : ").strip().lower()

    if choice == 'e':
      filename = input("File Path : ").strip()
      print("")

      # validate filename
      if not os.path.isfile(filename):
        print(f"File '{filename}' does not exist \n")
        continue

        # encrypt based on user
      if user == 'A':
        encrypt_file(filename, user, keyA)
      elif user == 'B':
        encrypt_file(filename, user, keyB)
      else:
        print("Invalid user \n")

    elif choice == 'd':
      filename = input("File Path: ").strip()
      print("")

      # validate filename
      if not os.path.isfile(filename):
        print(f"File '{filename}' does not exist \n")
        continue

      # decrypt files
      decrypt_file(filename, keyA, keyB, public_key)

    elif choice == 'q':
      print("Exiting the program \n")
      break

    else:
      print("Invalid choice \n")

    print("Signing Out User ", user)
    print("")

if __name__ == "__main__":
  main()

------------------------------------
*** FILE ENCRYPTION & DECRYPTION *** 
------------------------------------

Kyber Key Sharing 

--------------------------
*** User A Key Sharing ***
-------------------------- 

A generates public-private key pair 

A sends public key over the internet 

Public Key:  b'\xfex\xbab\x01S5\xd5\x9b\\[\x8fu\xa1\t\xcfi\x02|k\x10.\xabo\nC\x0c\xa3L\xc3\x0e\xf6\xcb\xba\x84\x80\xa5%\x97\xebRa!ia\x84\x17DQs\x8b`6\xbf]\xa6\xa1+,\x17\x84\xd5\x83JZ0\xf0Dj\x87x)\t\xc8\xb9x\xd3\x8e\xa28m\x85\xc0LO\x8ch,c"o\x99\xaf\xcd\n.\xd0\xf2\x1c\x9bk\xb1 "9zU\x96\xe4\x19\x0cq\xfb<E\xda\xc4\xf4\xfb\x86\xf9j\x0e\xc0\xe5\xa7\xcf\xb4(\xd0\xd6\x905\xe1\xc8,pN\xb7\xf7)\xfd{\xb8\xed\xb2\xb6\xf7\xf5\x19\xea\xbaE6\x08\x95\x05kX\xca@+\x0b\xdc5\xdf\x9c\xc2%\x87\x07\xa4I"\xfcR4r\xa0/\xabyIIu\x84\xd9\xa3\x1d\xc7\xba\x1ajQ\xa3K\xe7\x9f\xedV\xb1kY%~w\xad\xdf\xd8\x8a\xbcp\x91\x8c\x0b\x90\xdbiP\x85q\xbd\xe3\xf7\xcb\xdag\xbc\xc2P8$\xda\x04\x17\x93M\xe2\x10n\x16\xbb\xaa\x02\xb3,c\x88]Zt7\x1cz/%\x